# GitHub Marketplace मोडेल इन्फरेन्स Phi-4 Reasoning संग

यो नोटबुकले GitHub Marketplace मोडेलहरू इन्फरेन्सको लागि कसरी प्रयोग गर्ने भन्ने देखाउँछ, विशेष गरी Microsoft को Phi-4-reasoning मोडेल प्रयोग गर्दै।

## सेटअप र कन्फिगरेसन

पहिले, हामी हाम्रो वातावरण कन्फिगर गर्नेछौं र आवश्यक निर्भरता स्थापना गर्नेछौं।


In [ ]:
# Install required packages
!pip install requests python-dotenv
!pip install azure-ai-inference

### स्थानीय.env फाइल सेटअप गर्ने

यो नोटबुक चलाउनु अघि, तपाईंले यो नोटबुकको समान डाइरेक्टरीमा `local.env` नामको फाइल निम्न भेरिएबलहरू सहित बनाउनुपर्नेछ:

```
# GitHub Configuration
GITHUB_TOKEN=your_personal_access_token_here
GITHUB_INFERENCE_ENDPOINT=https://models.github.ai/inference
GITHUB_MODEL=microsoft/Phi-4-reasoning

# Azure OpenAI Configuration
AZURE_API_KEY=your_azure_api_key_here
AZURE_OPENAI_ENDPOINT=your_azure_endpoint_here
AZURE_OPENAI_MODEL=Phi-4-reasoning
```

**निर्देशनहरू:**

1. यो नोटबुकको समान फोल्डरमा `local.env` नामको नयाँ फाइल बनाउनुहोस्
2. माथि देखाइएका तीनवटा वातावरणीय भेरिएबलहरू थप्नुहोस्
3. `your_personal_access_token_here` लाई तपाईंको GitHub व्यक्तिगत पहुँच टोकनसँग बदल्नुहोस्
4. सानो मोडेलको लागि `microsoft/Phi-4-mini-reasoning` मा मोडेल परिवर्तन गर्न सक्नुहुन्छ

**Note:** GitHub टोकनलाई AI मोडेल सेवा पहुँच गर्न उपयुक्त अनुमति आवश्यक हुन्छ।


## वातावरण चरहरू लोड गर्नुहोस्

हामी हाम्रो वातावरण चरहरू `local.env` फाइलबाट लोड गर्नेछौं, जसमा हाम्रो GitHub टोकन र मोडेल जानकारी समावेश छ।


In [ ]:
import os
import requests
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Load variables from local.env file
load_dotenv('local.env')

# Access the environment variables - using values from local.env file
endpoint = os.getenv("GITHUB_INFERENCE_ENDPOINT")
model = os.getenv("GITHUB_MODEL")
token = os.getenv("GITHUB_TOKEN")
azuretoken = os.getenv("AZURE_KEY")
azureendpoint = os.getenv("AZURE_ENDPOINT")
azuremodel = os.getenv("AZURE_MODEL")
# Use fallback values if not found in local.env
if not endpoint:
    endpoint = "https://models.github.ai/inference"
    print("Warning: GITHUB_INFERENCE_ENDPOINT not found in local.env, using default value")

if not model:
    model = "microsoft/Phi-4-reasoning"
    print("Warning: GITHUB_MODEL not found in local.env, using default value")
    print("To change the model to Phi-4-mini-reasoning use \"microsoft/Phi-4-mini-reasoning\"")

if not token:
    raise ValueError("GITHUB_TOKEN not found in local.env file. Please add your GitHub token.")

print(f"Endpoint: {endpoint}")
print(f"Model: {model}")
print(f"azure_ai_image_generation_new.ipynb: {azureendpoint}")
print(f"azuremodel: {azuremodel}")
print(f"azuretoken available: {'Yes' if azuretoken else 'No'}")
print(f"Token available: {'Yes' if token else 'No'}")

## मोडेल अनुमानका लागि सहायक फङ्सनहरू

GitHub अनुमान API सँग अन्तरक्रिया गर्न सहायक फङ्सनहरू बनाउँ।


In [5]:
def generate_completion(prompt, model_id=model, temperature=0.7, max_tokens=10000):
    """Generate a completion using the GitHub inference API"""
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_id,
        "prompt": prompt,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        # GitHub Models API requires a different endpoint structure
        api_url = f"{endpoint}/v1/chat/completions"
        print(f"Calling API at: {api_url}")
        
        # Modify payload for chat completions format
        chat_payload = {
            "model": model_id,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": temperature,
            "max_tokens": max_tokens
        }
        
        response = requests.post(api_url, headers=headers, json=chat_payload)
        response.raise_for_status()  # Raise exception for 4XX/5XX errors
        result = response.json()
        
        if 'choices' in result and len(result['choices']) > 0:
            # Handle chat completions response format
            if 'message' in result['choices'][0] and 'content' in result['choices'][0]['message']:
                return result['choices'][0]['message']['content']
            # Fall back to the text field if available
            elif 'text' in result['choices'][0]:
                return result['choices'][0]['text']
            else:
                return f"Error: Could not extract content from response: {result['choices'][0]}"
        else:
            return f"Error: Unexpected response format: {result}"
    except Exception as e:
        print(f"Full error details: {str(e)}")
        return f"Error during API call: {str(e)}"

def format_conversation(messages):
    """Format a conversation for the model"""
    # For chat completion API, we'll just return the messages directly
    return messages

def generate_chat_completion(messages, model_id=model, temperature=0.7, max_tokens=1000):
    """Generate a completion using GitHub's chat completions API"""
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        api_url = f"{endpoint}/v1/chat/completions"
        print(f"Calling API at: {api_url}")
        
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        if 'choices' in result and len(result['choices']) > 0:
            if 'message' in result['choices'][0]:
                return result['choices'][0]['message']['content']
            else:
                return f"Error: Unexpected response format: {result['choices'][0]}"
        else:
            return f"Error: Unexpected response format: {result}"
    except Exception as e:
        print(f"Full error details: {str(e)}")
        return f"Error during API call: {str(e)}"
        
# For backward compatibility with existing code
def format_prompt_legacy(messages):
    """Format a conversation in text format (legacy method)"""
    formatted_prompt = ""
    
    for msg in messages:
        role = msg.get("role", "")
        content = msg.get("content", "")
        
        if role == "user":
            formatted_prompt += f"User: {content}\n\n"
        elif role == "assistant":
            formatted_prompt += f"Assistant: {content}\n\n"
        elif role == "system":
            formatted_prompt += f"{content}\n\n"
    
    formatted_prompt += "Assistant: "
    return formatted_prompt

## उदाहरण १: ९ वटा r का लागि कति स्ट्रबेरीहरू चाहिन्छ?

आउनुहोस्, स्ट्रबेरीहरू र r का बारेमा सोध्दै हाम्रो पहिलो अनुमान उदाहरण चलाउँ।


In [6]:
example1_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that answers questions accurately and concisely."},
    {"role": "user", "content": "How many strawberries do I need to collect 9 r's?"}
]

print("Messages:")
for msg in example1_messages:
    print(f"{msg['role']}: {msg['content']}")
print("\nGenerating response...\n")

# Use the new chat completion function directly with the messages
response1 = generate_chat_completion(example1_messages)
print("Response:")
print(response1)

Messages:
system: You are a helpful AI assistant that answers questions accurately and concisely.
user: How many strawberries do I need to collect 9 r's?

Generating response...

Calling API at: https://models.github.ai/inference/v1/chat/completions
Response:
<think>User says: "How many strawberries do I need to collect 9 r's?" This appears to be some riddle. Possibly the phrase "9 r's" might be a pun or reference to "r" letters. For example, "How many strawberries do I need to collect 9 r's?" It might be a pun on the phrase "strawberry, nine, r's" or "I need 9 r's" might be a riddle. Alternatively, perhaps the question "How many strawberries do I need to collect 9 r's?" might be a riddle where the answer is something like "9 strawberries" are needed if "strawberry" contains one "r" or two? Let's think: "Strawberry" letter count: S T R A W B E R R Y. Counting letter "r": "strawberry" has two "r"s, one in "straw" and one in "berry" but wait, let's check: "strawberry" letters: S, T, R, A

### उदाहरण १ को विश्लेषण

यस उदाहरणमा, मोडेलले बुझ्नुपर्छ कि "strawberry" शब्दमा दुईवटा 'r' अक्षरहरू छन्। त्यसैले, ९ 'r' अक्षरहरू जम्मा गर्नका लागि, तपाईंलाई ५ वटा strawberries (जसमा कुल १० 'r' हुन्छ) चाहिन्छ, वा ठीक ९ 'r' पाउनका लागि ४.५ वटा strawberries आवश्यक पर्छ।

अब हेरौं कि Phi-4-reasoning मोडेलले यो समस्यालाई कसरी समाधान गर्छ।


## उदाहरण २: पहेली समाधान गर्ने

अब हामी एउटा जटिल उदाहरण प्रयास गरौं - धेरै उदाहरणहरू भएको ढाँचाको पहेली।


In [7]:
example2_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that solves riddles and finds patterns in sequences."},
    {"role": "user", "content": "I will give you a riddle to solve with a few examples, and something to complete at the end"},
    {"role": "user", "content": "nuno Δημήτρης evif Issis 4"},
    {"role": "user", "content": "ntres Inez neves Margot 4"},
    {"role": "user", "content": "ndrei Jordan evlewt Μαρία 9"},
    {"role": "user", "content": "nπέντε Kang-Yuk xis-ytnewt Nubia 21"},
    {"role": "user", "content": "nπέντε Κώστας eerht-ytnewt Μανώλης 18"}, 
    {"role": "user", "content": "nminus one-point-two Satya eno Bill X."},
    {"role": "user", "content": "What is a likely completion for X that is consistent with examples above?"}
]

print("Messages:")
for msg in example2_messages:
    print(f"{msg['role']}: {msg['content'][:50]}...")
print("\nGenerating response...\n")

response2 = generate_chat_completion(example2_messages, temperature=0.2, max_tokens=10000)
print("Response:")
print(response2)

Messages:
system: You are a helpful AI assistant that solves riddles...
user: I will give you a riddle to solve with a few examp...
user: nuno Δημήτρης evif Issis 4...
user: ntres Inez neves Margot 4...
user: ndrei Jordan evlewt Μαρία 9...
user: nπέντε Kang-Yuk xis-ytnewt Nubia 21...
user: nπέντε Κώστας eerht-ytnewt Μανώλης 18...
user: nminus one-point-two Satya eno Bill X....
user: What is a likely completion for X that is consiste...

Generating response...

Calling API at: https://models.github.ai/inference/v1/chat/completions
Response:
<think>We are given a riddle with examples. The riddle is: "I will give you a riddle to solve with a few examples, and something to complete at the end". The examples are:

1. "nuno Δημήτρης evif Issis 4"
2. "ntres Inez neves Margot 4"
3. "ndrei Jordan evlewt Μαρία 9"
4. "nπέντε Kang-Yuk xis-ytnewt Nubia 21"
5. "nπέντε Κώστας eerht-ytnewt Μανώλης 18"
6. "nminus one-point-two Satya eno Bill X."

We are asked: "What is a likely completion for X that is

### उदाहरण २ को विश्लेषण

यो पहेलीले विभिन्न भाषाहरू र संख्यात्मक प्रतिनिधित्वहरूमा जटिल ढाँचाहरू पहिचान गर्न आवश्यक छ। आउनुहोस्, मोडेलले बुझ्नुपर्ने कुराहरूलाई टुक्रा-टुक्रामा विश्लेषण गरौं:

1. शब्दहरूमा उल्टो हिज्जे जस्तै "evif" (five) पहिचान गर्नुहोस्
2. विभिन्न भाषाहरूमा संख्याहरू चिन्हित गर्नुहोस् (जस्तै, स्पेनिशमा "uno", ग्रीकमा "πέντε")
3. संख्याहरू र अन्तिम अंकको बीचको सम्बन्ध पत्ता लगाउनुहोस्

ढाँचाले विभिन्न भाषाहरू र स्वरूपहरूमा प्रतिनिधित्व गरिएका मानहरू बीच गणितीय क्रियाहरू समावेश गरेको देखिन्छ।


## विभिन्न प्यारामिटरहरूसँग प्रयोग गर्दै

आउनुहोस् दोस्रो उदाहरणलाई फेरि फरक तापक्रम सेटिङहरूसँग प्रयास गरौं ताकि यसले मोडेलको प्रतिक्रिया कसरी प्रभावित गर्छ हेर्न सकियोस्।


In [ ]:
# Try with a higher temperature for more creative responses
response_creative = generate_chat_completion(example2_messages, temperature=0.9, max_tokens=20000)
print("Response with higher temperature (0.9):")
print(response_creative)

## आफ्नो उदाहरण बनाउनुहोस्

तपाईं मोडेलको तर्क क्षमताहरू परीक्षण गर्न आफ्नै उदाहरणहरू बनाउन सक्नुहुन्छ। तलका प्रॉम्प्टहरू परिमार्जन गर्ने वा पूर्ण रूपमा नयाँ परिदृश्यहरू सिर्जना गर्ने प्रयास गर्नुहोस्।


In [ ]:
# Define your custom prompt here
custom_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that can solve complex problems."},
    {"role": "user", "content": "Your custom prompt here"}
]

# Uncomment the lines below to run your custom prompt
# custom_response = generate_chat_completion(custom_messages)
# print("Response to custom prompt:")
# print(custom_response)

## निष्कर्ष

यस नोटबुकले GitHub Marketplace मोडेलहरूलाई अनुमानको लागि कसरी प्रयोग गर्ने भन्ने देखायो, विशेष गरी Phi-4-reasoning मोडेल प्रयोग गरेर तार्किक समस्याहरू र पहेलिहरू समाधान गर्न।

मुख्य सिकाइहरू:
1. GitHub टोकनहरूसँग प्रमाणिकरण सेटअप गर्नु
2. उत्तम अनुमानको लागि प्रम्प्टहरू ढाँचा बनाउनु
3. तापक्रम जस्ता मोडेल प्यारामिटरहरू व्यवस्थापन गरेर प्रतिक्रिया भिन्नता नियन्त्रण गर्नु
4. विभिन्न प्रकारका समस्याहरूमा मोडेलको तर्क क्षमताहरू परीक्षण गर्नु

आफ्नो GitHub टोकन सुरक्षित राख्न र यसलाई सार्वजनिक रिपोजिटरीहरू वा नोटबुकहरूमा साझा नगर्न सम्झनुहोस्।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
